In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

# Read the Excel file into a pandas DataFrame
df = pd.read_excel('dataset.xlsx')
df['KOSTENRUBRIEK - declared'] = df['KOSTENRUBRIEK - declared'].str.lower()
df['FLC: REDEN VERWERPING'] = df['FLC: REDEN VERWERPING'].str.lower()
df['BESCHRIJVING DECLARATIE'] = df['BESCHRIJVING DECLARATIE'].str.lower()

Unique flags

In [2]:
# Extract unique flags from the "FLC: REDEN VERWERPING" column
all_flags = df["FLC: REDEN VERWERPING"].str.split('|', expand=True).stack().str.strip().unique()

# Print the unique flags
print("Unique Flags:")
for flag in all_flags:
    print(flag)

Unique Flags:
the cost was declared in the wrong category
double declaration
please specify the correct invoiced amount
costs that are declared too late, can only be reimbursed for 50%
purchase not eligible for subsidy
please specify the correct supplier


In [3]:
# Define the phrases to check for in the specified column
phrases = {
    "FLC: REDEN VERWERPING": {
        "double declaration": "double",
        "too late": "late",
        "correct invoiced amount": "amount",
        "supplier": "supplier",
        "wrong category": "category",
        "eligible": "eligible"
    }
}

# Iterate through the phrases and update the corresponding columns
for column_name, conditions in phrases.items():
    for phrase, new_column in conditions.items():
        df[new_column] = df[column_name].str.contains(phrase, case=False, na=False).astype(int)

Double

In [4]:
df['pred.double'] = 0

# Check if an entry has the same values in both columns 'REFERENTIE FACTUUR' and 'DATUM FACTUUR - DECLARED'
duplicate_entries = df[df.duplicated(['REFERENTIE FACTUUR', 'DATUM FACTUUR - DECLARED'], keep=False)]

for idx, group in duplicate_entries.groupby(['REFERENTIE FACTUUR', 'DATUM FACTUUR - DECLARED']):
    # Find the earliest value in 'DECLARATIEDATUM (can be assumed to be close to payment date)' for this entry
    earliest_date = group['DECLARATIEDATUM (can be assumed to be close to payment date)'].min()
    
    # Set 'pred.double' to 1 for entries other than the one with the earliest date
    df.loc[group.index, 'pred.double'] = (group['DECLARATIEDATUM (can be assumed to be close to payment date)'] != earliest_date).astype(int)

Too late

In [5]:
# Convert the "DATUM FACTUUR - DECLARED" column to datetime format
df['DATUM FACTUUR - DECLARED'] = pd.to_datetime(df['DATUM FACTUUR - DECLARED'], errors='coerce')

# Create the "latest" column by moving the month 6 months forward
df['latest'] = df['DATUM FACTUUR - DECLARED'] + pd.DateOffset(months=6)

# Convert date columns to datetime objects
df['latest'] = pd.to_datetime(df['latest'])
df['DECLARATIEDATUM (can be assumed to be close to payment date)'] = pd.to_datetime(df['DECLARATIEDATUM (can be assumed to be close to payment date)'])

# Calculate the time difference and create a new column 'timediff'
df['pred.late'] = (df['DECLARATIEDATUM (can be assumed to be close to payment date)'] - df['latest']).dt.days > 0
df['pred.late'] = df['pred.late'].astype(int)

Wrong amount (only EUR works)

In [6]:
df['pred.amount'] = (df['BETAALD BEDRAG - extracted from invoice'] - df['BETAALD BEDRAG - declared ']).apply(lambda x: 1 if x < 0 else 0)

Wrong supplier

In [7]:
se = SnowballStemmer("english")

x = df["LEVERANCIER - EXTRACTED"]
y = df["LEVERANCIER - DECLARED"]

def clean_message(message_list):
    ms = []
    for word in message_list:
        ms.append(se.stem(word))
    return ms 

def lever_equals(x, y):
    if x == y:
        return 0
    x = re.sub(r'[^\w\s]', ' ', x)
    y = re.sub(r'[^\w\s]', ' ', y)
    split_x = clean_message(word_tokenize(x.lower()))
    split_y = clean_message(word_tokenize(y.lower()))
    
    if len(split_x) == len(split_y):
        for i in range(len(split_x)):
            if split_x[i][0] != split_y[i][0]:
                return 1
        return 0
    elif len(split_x)!=0 and len(split_y)!=0:
        mini = min(len(split_x), len(split_y))
        for i in range(mini):
            if split_x[i] != split_y[i]:
                return 1
        return 0
    return 1

# Create a new column 'pred.supplier' in df and store the results
df['pred.supplier'] = [lever_equals(x[i], y[i]) for i in range(len(x))]


Wrong category

In [8]:
# Filter rows where "category" column has value 0
df2 = df[df['category'] == 0].copy()  # Make a copy of the filtered DataFrame

# Save the filtered DataFrame to a CSV file
df2.to_csv('cat_data.csv', index=False)

# Train model on positive targets
X = df2['BESCHRIJVING DECLARATIE']
y = df2['KOSTENRUBRIEK - declared']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a Multinomial Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)

# Make predictions on the training set
train_predictions = classifier.predict(X_train_tfidf)

# Update 'pred.category' in the original DataFrame for the training set
df2.loc[df2['BESCHRIJVING DECLARATIE'].index, 'pred.category'] = (train_predictions != y_train).astype(int)

# Make predictions on the test set
test_predictions = classifier.predict(X_test_tfidf)

# Update 'pred.category' in the original DataFrame for the test set
df2.loc[X_test.index, 'pred.category'] = (test_predictions != y_test).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, test_predictions)
classification_rep = classification_report(y_test, test_predictions)

print(f'Accuracy: {accuracy:.2f}')
print('\nClassification Report:')
print(classification_rep)


Accuracy: 0.92

Classification Report:
                             precision    recall  f1-score   support

                  equipment       0.95      0.95      0.95        19
          external services       0.87      0.91      0.89        22
infrastructure and building       0.94      0.94      0.94        16
          preparation costs       0.90      0.82      0.86        22
               travel costs       0.94      1.00      0.97        16

                   accuracy                           0.92        95
                  macro avg       0.92      0.92      0.92        95
               weighted avg       0.92      0.92      0.92        95



In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Filter rows where "category" column has value 0
df2 = df[df['category'] == 0].copy()  # Make a copy of the filtered DataFrame

# Train model on positive targets
X = df2['BESCHRIJVING DECLARATIE']
y = df2['KOSTENRUBRIEK - declared']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a Multinomial Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)

# Make predictions on the training set
train_predictions = classifier.predict(X_train_tfidf)
train_probabilities = classifier.predict_proba(X_train_tfidf)

# Save the probabilities in separate columns for each category in the original DataFrame for the training set
for i, class_label in enumerate(classifier.classes_):
    df2[f'pred.probability_{class_label}'] = train_probabilities[:, i]

# Make predictions on the test set
test_predictions = classifier.predict(X_test_tfidf)
test_probabilities = classifier.predict_proba(X_test_tfidf)

# Save the probabilities in separate columns for each category in the original DataFrame for the test set
for i, class_label in enumerate(classifier.classes_):
    df2[f'pred.probability_{class_label}'] = test_probabilities[:, i]

# Update 'pred.category' in the original DataFrame for the training set
df2.loc[df2['BESCHRIJVING DECLARATIE'].index, 'pred.category'] = (train_predictions != y_train).astype(int)

# Update 'pred.category' in the original DataFrame for the test set
df2.loc[X_test.index, 'pred.category'] = (test_predictions != y_test).astype(int)

# Save the DataFrame to a CSV file
df2.to_csv('cat_data_with_probabilities.csv', index=False)

# Evaluate the model
accuracy = accuracy_score(y_test, test_predictions)
classification_rep = classification_report(y_test, test_predictions)

print(f'Accuracy: {accuracy:.2f}')
print('\nClassification Report:')
print(classification_rep)


ValueError: Length of values (377) does not match length of index (472)

In [ ]:
# Use model to predict on positive targets
df_category_1 = df[df['category'] == 1].copy()  # Make a copy to avoid modifying the original DataFrame

# Extract text data and labels
X_category_1 = df_category_1['BESCHRIJVING DECLARATIE']
y_category_1 = df_category_1['KOSTENRUBRIEK - declared']

# Vectorize the text data using the same TF-IDF vectorizer
X_category_1_tfidf = vectorizer.transform(X_category_1)

# Make predictions on the instances where 'category' has value 1
predictions_category_1 = classifier.predict(X_category_1_tfidf)

# Update 'pred.category' in the original DataFrame
df_category_1['pred.category'] = 1-(predictions_category_1 == y_category_1).astype(int)

# Evaluate the model on these instances
accuracy_category_1 = accuracy_score(y_category_1, predictions_category_1)
classification_rep_category_1 = classification_report(y_category_1, predictions_category_1)

print(f'Accuracy: {1 - accuracy_category_1:.2f}')
print('\nClassification Report:')
print(classification_rep_category_1)


Accuracy: 0.95

Classification Report:
                             precision    recall  f1-score   support

                  equipment       0.00      0.00      0.00         8
          external services       0.00      0.00      0.00         8
infrastructure and building       0.00      0.00      0.00         3
          preparation costs       0.25      0.50      0.33         2
               travel costs       0.00      0.00      0.00         1

                   accuracy                           0.05        22
                  macro avg       0.05      0.10      0.07        22
               weighted avg       0.02      0.05      0.03        22



In [ ]:
df = pd.concat([df2, df_category_1], ignore_index=True)

Not eligible

In [ ]:
# Load the data from the Excel file
df_eli = pd.read_excel('eli_data.xlsx')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_eli['BESCHRIJVING DECLARATIE'], df_eli['eligible'], test_size=0.2, random_state=42)

# Create a pipeline with TF-IDF vectorizer, stopwords removal, and Decision Tree classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),  # Using TF-IDF vectorizer with stopwords removal
    ('classifier', DecisionTreeClassifier(random_state=42, class_weight="balanced"))
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model on the test set
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy:.2f}")

# Assuming 'df' is your DataFrame for testing
# Make predictions on the 'df' DataFrame
df['pred.eligible'] = pipeline.predict(df['BESCHRIJVING DECLARATIE'])

Accuracy on the test set: 0.98


In [ ]:
# Save the updated DataFrame to a new Excel file
output_file_path = 'output_file.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Updated Excel file saved to: {output_file_path}")


Updated Excel file saved to: output_file.xlsx


In [ ]:
# Extract the actual values
y_true = df[['double', 'late', 'amount', 'supplier', 'category', 'eligible']].values

# Extract the predicted values
y_pred = df[['pred.double', 'pred.late', 'pred.amount', 'pred.supplier', 'pred.category', 'pred.eligible']].values

# Calculate classification report
classification_rep = classification_report(y_true, y_pred, target_names=['double', 'late', 'amount', 'supplier', 'category', 'eligible'])
print("Classification Report:")
print(classification_rep)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true.flatten(), y_pred.flatten())
print("\nConfusion Matrix:")
print(conf_matrix)


Classification Report:
              precision    recall  f1-score   support

      double       1.00      0.40      0.57         5
        late       1.00      0.98      0.99        50
      amount       1.00      0.39      0.56        18
    supplier       0.62      0.89      0.73         9
    category       0.54      0.95      0.69        22
    eligible       1.00      1.00      1.00         6

   micro avg       0.80      0.85      0.82       110
   macro avg       0.86      0.77      0.76       110
weighted avg       0.88      0.85      0.82       110
 samples avg       0.16      0.16      0.16       110


Confusion Matrix:
[[2831   23]
 [  17   93]]


c:\Users\camd1\miniconda3\envs\holyhack\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\camd1\miniconda3\envs\holyhack\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\camd1\miniconda3\envs\holyhack\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resul

In [ ]:
# Initialize dictionaries to store accuracy rates and proportions of wrong predictions
accuracy_rates = {}
proportions_of_wrongs = {}

# Iterate over each feature
for feature in ['double', 'late', 'amount', 'supplier', 'category', 'eligible']:
    # Extract the actual values for the current feature
    y_true_feature = df[feature].values
    
    # Extract the predicted values for the current feature
    y_pred_feature = df[f'pred.{feature}'].values
    
    # Calculate accuracy rate for the current feature
    accuracy = accuracy_score(y_true_feature, y_pred_feature)
    
    # Store accuracy rate in the dictionary
    accuracy_rates[feature] = accuracy
    
    # Calculate proportion of wrong predictions for the current feature
    wrong_predictions = (y_true_feature != y_pred_feature).sum()
    total_predictions = len(y_true_feature)
    proportion_of_wrongs = wrong_predictions / total_predictions
    
    # Store mistakes predictions in the dictionary
    proportions_of_wrongs[feature] = proportion_of_wrongs

# Print accuracy rates
print("Accuracy Rates:")
for feature, accuracy in accuracy_rates.items():
    print(f"{feature}: {accuracy:.4f}")

# Print mistakes
print("\nMistakes:")
for feature, proportion_of_wrongs in proportions_of_wrongs.items():
    print(f"{feature}: {proportion_of_wrongs:.4f}")


Accuracy Rates:
double: 0.9939
late: 0.9980
amount: 0.9777
supplier: 0.9879
category: 0.9615
eligible: 1.0000

Mistakes:
double: 0.0061
late: 0.0020
amount: 0.0223
supplier: 0.0121
category: 0.0385
eligible: 0.0000
